# Week 3 Coursera Capstone Assignment Part 1

#### Import Libraries

In [1]:
import numpy as np
import pandas as pd
! pip install geocoder
import geocoder as gc
import requests
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
!conda install -c conda-forge folium=0.5.0
import folium
import random
print('Imports Completed')

Requirement not upgraded as not directly required: geocoder in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: ratelim in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: decorator in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ratelim->geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,

#### Read Wikipedia Table as Pandas Dataframe

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
canData = pd.read_html(url)
canData = canData[0] #Select first dataframe of the page

## Format Dataframe

In [3]:
canData.columns = ['Postal Code','Borough','Neighborhood'] #Rename Columns
canData = canData.drop([0]) #Remove extraneous header from df  

#### Remove or Replace 'Not assigned' values

In [4]:
#remove 'Not Assigned' from Boroughs
canData['Borough'].replace('Not assigned',np.nan,inplace = True)
canData.dropna(inplace=True)

#Replace 'Not assigned' Neighborhoods with Borough value
canData.reset_index(drop = True,inplace = True)  
for x in canData.index:  
    if (canData.loc[x]['Neighborhood'] == 'Not assigned'):  
        canData.loc[x]['Neighborhood'] = canData.loc[x]['Borough']

#### Combine Neighborhoods with same PostalCode

In [5]:
droplist = []  
for x in range(1,len(canData.index.unique()-1)):  
    if (canData.loc[x-1]['Postal Code'] == canData.loc[x]['Postal Code']):  
        canData.loc[x]['Neighborhood'] = canData.loc[x]['Neighborhood'] + (', '+canData.loc[x-1]['Neighborhood'])  
        droplist.append(x-1)  
canData.drop(droplist, inplace=True)  

## Display Dataframe and Number of Rows

In [6]:
canData.sort_values(by='Postal Code',inplace=True)
canData.reset_index(drop = True,inplace = True)
print('Number of Rows:',canData.shape[0])
canData

Number of Rows: 103


,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek"
2,M1E,Scarborough,"West Hill, Morningside, Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Oakridge, Golden Mile, Clairlea"
8,M1M,Scarborough,"Scarborough Village West, Cliffside, Cliffcrest"
9,M1N,Scarborough,"Cliffside West, Birch Cliff"


## Retrieve Coordinates

In [7]:
coord = pd.read_csv('http://cocl.us/Geospatial_data')
canData = canData.merge(coord, how = 'right',on = 'Postal Code')

## Generate Toronto Map

In [8]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(address,'\nLatitude:',latitude,'\nLongitude:',longitude)

Toronto, ON 
Latitude: 43.653963 
Longitude: -79.387207


In [9]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)

#add labels
for lat, lng, borough, neighborhood in zip(canData['Latitude'], canData['Longitude'], canData['Borough'], canData['Neighborhood']):
    label = '{}: {}'.format(borough, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)

#display map
toronto_map

## Explore Neighborhoods

#### Set Foursquare Credentials

In [10]:
CLIENT_ID = 'V5GTPEARUI4LNNVRL2Z2CBTSKQX32TZB14ZUP4KKDB0DS01L' # your Foursquare ID
CLIENT_SECRET = 'FNNRQJQGSE23SQVTQ1MX0DKED01IW5B5O50H0R5K3CKFEBWL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: V5GTPEARUI4LNNVRL2Z2CBTSKQX32TZB14ZUP4KKDB0DS01L
CLIENT_SECRET:FNNRQJQGSE23SQVTQ1MX0DKED01IW5B5O50H0R5K3CKFEBWL


#### Select Boroughs Containing 'Toronto'

In [11]:
torData = canData[canData['Borough'].str.contains('Toronto')]

#### Define function to request foursquare api

In [12]:
def getVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Get 100 Venues of Each Postal Code Area

In [13]:
toronto_venues = getVenues(torData['Neighborhood'], torData['Latitude'], torData['Longitude'])

The Beaches
Riverdale, The Danforth West
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Summerhill East, Moore Park
Summerhill West, South Hill, Rathnelly, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, King, Adelaide
Union Station, Toronto Islands, Harbourfront East
Toronto Dominion Centre, Design Exchange
Victoria Hotel, Commerce Court
Roselawn
Forest Hill West, Forest Hill North
Yorkville, North Midtown, The Annex
University of Toronto, Harbord
Kensington Market, Grange Park, Chinatown
South Niagara, Railway Lands, King and Spadina, Harbourfront West, Island airport, Bathurst Quay, CN Tower
Stn A PO Boxes 25 The Esplanade
Underground city, First Canadian Place
Christie
Dufferin, Dovercourt Village
Trinity, Little Portugal
Parkdale Village, Exhibition Place, Brockton
The Junction So

## Cluster Neighborhoods

#### Encode Venue Types

In [14]:
toronto_venues
# one hot encoding
toronto_encode = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_encode['Area'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_encode.columns[-1]] + list(toronto_encode.columns[:-1])
toronto_encode = toronto_encode[fixed_columns]

#### Get Top 5 Venues for each Neighborhood

In [15]:
#calculate mean venue type for each neighborhood
toronto_sorted = toronto_encode.groupby(by='Area').mean().reset_index()

In [38]:
#Create empty dataframe to be filled
topVen = pd.DataFrame()
#Get top 5 venues for each Area
for count, a in enumerate(toronto_sorted['Area']):
    temp = toronto_sorted[toronto_sorted['Area'] == a]
    temp = temp.drop('Area',axis=1)
    temp.reset_index()
    temp = temp.sort_values(by=count,axis=1,ascending=False)
    topVen = topVen.append(pd.Series(temp.iloc[:,0:5].columns),ignore_index = True)

In [39]:
#Rename Columns & Display first 5 rows
topVen.columns = ['#1 Venue','#2 Venue','#3 Venue','#4 Venue','#5 Venue'] 
topVen.head()

,#1 Venue,#2 Venue,#3 Venue,#4 Venue,#5 Venue
0,Coffee Shop,Cocktail Bar,Restaurant,Café,Beer Bar
1,Light Rail Station,Yoga Studio,Restaurant,Brewery,Pizza Place
2,Coffee Shop,Café,Italian Restaurant,Burger Joint,Japanese Restaurant
3,Café,Grocery Store,Park,Coffee Shop,Baby Store
4,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant


#### Create Final Data frame with Neighborhoods, Cluster Labels (k=4), and Top 5 Venues

In [40]:
#fit toronto data to kmeans
toronto_cluster = toronto_sorted.drop(labels='Area',axis=1)
kclusters = 4
kmeans = KMeans(kclusters, random_state=0).fit(toronto_cluster)

In [41]:
topVen.insert(0,'Cluster Labels',kmeans.labels_)

In [42]:
torFinal = torData.sort_values(by='Neighborhood').reset_index(drop=True).join(topVen)
torFinal

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,#1 Venue,#2 Venue,#3 Venue,#4 Venue,#5 Venue
0,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Restaurant,Café,Beer Bar
1,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,0,Light Rail Station,Yoga Studio,Restaurant,Brewery,Pizza Place
2,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Café,Italian Restaurant,Burger Joint,Japanese Restaurant
3,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Café,Grocery Store,Park,Coffee Shop,Baby Store
4,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant
5,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Pizza Place,Sandwich Place,Dessert Shop,Café,Italian Restaurant
6,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Breakfast Spot,Hotel,Gym,Park,Food & Drink Shop
7,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,0,Pharmacy,Bakery,Supermarket,Brazilian Restaurant,Bar
8,M5P,Central Toronto,"Forest Hill West, Forest Hill North",43.696948,-79.411307,3,Trail,Sushi Restaurant,Bus Line,Jewelry Store,Adult Boutique
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant


## Map Clusters

In [46]:
#initialize map
clusterMap = folium.Map(location=[latitude, longitude], zoom_start=11)

colors = ['blue','red','green','black']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(torFinal['Latitude'], torFinal['Longitude'], torFinal['Neighborhood'], torFinal['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colors[cluster],
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(clusterMap)
clusterMap